In [241]:
import pandas as pd
import numpy as np
import re
import json

In [242]:
df = pd.read_csv("../movies_dataset.csv")

C:\Users\Pablo\AppData\Local\Temp\ipykernel_5372\3976793484.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../movies_dataset.csv")


In [243]:
df.shape

(45466, 24)

In [244]:
df.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [245]:
df.isna().sum()

adult                        0
belongs_to_collection    40972
budget                       0
genres                       0
homepage                 37684
id                           0
imdb_id                     17
original_language           11
original_title               0
overview                   954
popularity                   5
poster_path                386
production_companies         3
production_countries         3
release_date                87
revenue                      6
runtime                    263
spoken_languages             6
status                      87
tagline                  25054
title                        6
video                        6
vote_average                 6
vote_count                   6
dtype: int64

In [246]:
df["belongs_to_collection"].sample(15)

7231     NaN
25623    NaN
14198    NaN
853      NaN
31505    NaN
35183    NaN
39496    NaN
30640    NaN
4198     NaN
18835    NaN
42494    NaN
17739    NaN
27901    NaN
41347    NaN
14711    NaN
Name: belongs_to_collection, dtype: object

#### Tratamiento sobre columnas que sean colecciones

In [247]:
coleccion = df['belongs_to_collection']

In [248]:
id = []
name = []
poster_path = []
backdrop_path = []
for linea in coleccion:
    try:
        lista_partes = linea.split(',')
        id.append(lista_partes[0][7:])
        name.append(lista_partes[1][10:-1])
        poster_path.append(lista_partes[2][18:-1])
        backdrop_path.append(lista_partes[3][20:-2])
        pass
    except:
        pass
dic = {'id':id[:-3],
       'name':name,
       'poster_path':poster_path,
       'backdrop_path':backdrop_path}
id = np.array(id[:-3])
name = np.array(name)
poster_path = np.array(poster_path)
backdrop_path = np.array(backdrop_path)
df_belongs_to_collection = pd.DataFrame(data=dic)

In [249]:
df_belongs_to_collection

,id,name,poster_path,backdrop_path
0,10194,Toy Story Collection,7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg,9FBwqcd9IRruEDUrTdcaafOMKUq.jpg
1,119050,Grumpy Old Men Collection,nLvUdqgPgm3F85NMCii9gVFUcet.jpg,hypTnLot2z8wpFS7qwsQHW1uV8u.jpg
2,96871,Father of the Bride Collection,nts4iOmNnq7GNicycMJ9pSAn204.jpg,7qwE57OVZmMJChBpLEbJEmzUydk.jpg
3,645,James Bond Collection,HORpg5CSkmeQlAolx3bKMrKgfi.jpg,6VcVl48kNKvdXOZfJPdarlUGOsk.jpg
4,117693,Balto Collection,w0ZgH6Lgxt2bQYnf1ss74UvYftm.jpg,9VM5LiJV0bGb1st1KyHA3cVnO2G.jpg
...,...,...,...,...
4486,152918,The Carry On Collection,2P0HNrYgKDvirV8RCdT1rBSJdbJ.jpg,38tF1LJN7ULeZAuAfP7beaPMfcl.jpg
4487,52888,The Carry On Collection,2P0HNrYgKDvirV8RCdT1rBSJdbJ.jpg,38tF1LJN7ULeZAuAfP7beaPMfcl.jpg
4488,123720,The Carry On Collection,2P0HNrYgKDvirV8RCdT1rBSJdbJ.jpg,38tF1LJN7ULeZAuAfP7beaPMfcl.jpg
4489,37261,DC Super Hero Girls Collection,n,n


### codigo que anda

In [261]:
df = pd.read_csv("../movies_dataset.csv")

C:\Users\Pablo\AppData\Local\Temp\ipykernel_5372\3976793484.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../movies_dataset.csv")


In [262]:
def convertir_colecciones(df, columnas):
    for column in columnas:
        df[column] = df[column].apply(lambda x: ast.literal_eval(x) if pd.notnull(x) else [])
        df = df.explode(column)
        col_df = pd.json_normalize(df[column])
    return(col_df)

columnas_colecciones = ['genres', 'production_companies', 'production_countries', 'spoken_languages','belongs_to_collection']
df_desglosado = convertir_colecciones(df,columnas_colecciones)
df_desglosado

""
0
1
2
3
4
...
582881
582882
582883
582884


In [253]:
def expandcolumns(df, columns):
    for column in columns:
        df[column] = df[column].apply(lambda x: ast.literal_eval(x) if pd.notnull(x) else [])
        # Expandir la columna en filas
        df = df.explode(column)
        # Normalizar la columna expandida
        col_df = pd.json_normalize(df[column])
        # Renombrar las columnas normalizadas para evitar superposiciones
        col_df = col_df.add_prefix(f'{column}')
        # Concatenar las columnas normalizadas con el DataFrame original
        df = df.drop(columns=[column]).reset_index(drop=True).join(col_df)
    return df

In [254]:
columns_to_expand = ['genres', 'production_companies', 'production_countries', 'spoken_languages','belongs_to_collection']

In [ ]:
df_expand = expandcolumns(df,columns_to_expand)

In [258]:
print(df_expand.shape)
df_expand.isna().sum()

(45436, 27)


adult                                 0
budget                                0
homepage                          37659
id                                    0
imdb_id                              17
original_language                    11
original_title                        0
overview                            954
popularity                            5
poster_path                         386
release_date                         87
revenue                               6
runtime                             263
status                               87
tagline                           25035
title                                 6
video                                 6
vote_average                          6
vote_count                            6
genresid                           2442
genresname                         2442
production_companiesname          11874
production_companiesid            11874
production_countriesiso_3166_1     6285
production_countriesname           6285


In [ ]:
df_expand.shape

(582886, 27)

In [ ]:
df_expand.drop_duplicates(inplace=True,subset='id')
df_expand.shape

(45436, 27)

In [ ]:
df_creditos = pd.read_csv("../credits.csv")

def expandcolumns(df, columns):
    for column in columns:
        df[column] = df[column].apply(lambda x: ast.literal_eval(x) if pd.notnull(x) else []) #Convierte el contenido de cada celda en la columna column de df de una cadena de texto que representa una lista de diccionarios a una lista de diccionarios Python, utilizando ast.literal_eval. Si el valor es nulo (pd.notnull(x)), se devuelve una lista vacía.
        # Expande la columna column en filas, de modo que cada elemento de la lista de diccionarios se convierte en una fila separada en df
        df = df.explode(column)
        # Normaliza la columna column después de expandirla, lo que significa que convierte cada diccionario dentro de la lista en una serie de columnas.
        col_df = pd.json_normalize(df[column])
        # Renombra las columnas normalizadas agregando el nombre original de la columna como prefijo, para evitar conflictos si hay nombres de columnas duplicados en diferentes listas de diccionarios.
        col_df = col_df.add_prefix(f'{column}')
        # Elimina la columna original column del DataFrame df y la reemplaza con las columnas normalizadas (col_df), asegurándose de que el índice del DataFrame se reinicie para evitar problemas con el índice de las filas.
        df = df.drop(columns=[column]).reset_index(drop=True).join(col_df)
    return df

columns_to_expand = ['cast', 'crew']


df_credit_norm = expandcolumns(df_creditos, columns_to_expand)

In [ ]:
df_credit_norm.shape

(9484668, 16)

In [ ]:
df_credit_norm.sample(7)

,id,castcast_id,castcharacter,castcredit_id,castgender,castid,castname,castorder,castprofile_path,crewcredit_id,crewdepartment,crewgender,crewid,crewjob,crewname,crewprofile_path
1888306,29343,22.0,Tommy Hill,52fe45d8c3a368484e07247d,0.0,103575.0,Breck Costin,8.0,None,52fe45d9c3a368484e0724ad,Directing,1.0,194568.0,Director,Barbara Peeters,None
6289665,127585,157.0,Secret Service Agent,57f87f8fc3a368311a001643,0.0,105496.0,Neil Napier,62.0,/irI0HpejL0ugHpyQxvUJ665bhaU.jpg,5461db1d0e0a2672a3001c27,Art,1.0,1384362.0,Set Designer,Lucie Tremblay,None
5121433,60309,25.0,Abraham Lincoln,52fe461fc3a368484e0801e1,0.0,234683.0,Gerald Bestrom,17.0,None,52fe461fc3a368484e08017b,Writing,2.0,230882.0,Screenplay,James D. Solomon,None
328171,289,125.0,Emile - Waiter (uncredited),55eb1ff79251413e3500ac93,2.0,29274.0,Leo White,99.0,/wzD5hXyaVXQol86cYGGp3SOzlaq.jpg,52fe4233c3a36847f800b88f,Camera,2.0,2005.0,Director of Photography,Arthur Edeson,/6sjR4iLB5yKlJDPfIxdEey4yVp2.jpg
2062590,42149,12.0,Bill Richamn,52fe45f0c3a36847f80e32e1,2.0,150213.0,Nicholas Hormann,8.0,None,534adf7ec3a3682dfe001b7d,Crew,0.0,101427.0,Makeup Effects,Thomas R. Burman,None
850849,9314,19.0,Syme,52fe44e6c3a36847f80b0a85,2.0,208920.0,James Walker,5.0,/spCx7vRlcYpVDWGKhSM50kMNePt.jpg,589342db9251416ef80060ce,Art,2.0,49345.0,Assistant Art Director,Mark Raggett,None
492059,9549,25.0,Wally Schirra,52fe4508c3a36847f80b8543,2.0,2714.0,Lance Henriksen,6.0,/wf4Pr9RxsHGd0O9fLPiB3Al8IVC.jpg,575f48d7c3a36849e7000192,Production,2.0,1263.0,Casting,Lynn Stalmaster,/rhyzo2qgsGAXIpGTN5OabbZU95U.jpg


In [ ]:
import pandas as pd

# Ejemplo de DataFrame con estructura de diccionario en una columna
data = {'ID': [1, 2],
        'Nombres': [{'nombre': 'Juan', 'edad': 30}, {'nombre': 'Ana', 'edad': 25}]}
df = pd.DataFrame(data)

def expandir_nombres(nombres):
    for nombre in nombres:
        df[f'nombre_{nombre["nombre"]}'] = nombre["nombre"]
        df[f'edad_{nombre["nombre"]}'] = nombre["edad"]

df['Nombres'].apply(expandir_nombres)

print(df)


TypeError: string indices must be integers, not 'str'

In [ ]:
import ast

# String del diccionario
diccionario_str = "{'clave1': 'valor1', 'clave2': 'valor2'}"

# Convertir el string en un diccionario
diccionario = ast.literal_eval(diccionario_str)

print(diccionario)
print(type(diccionario))  # Para comprobar que es un diccionario

{'clave1': 'valor1', 'clave2': 'valor2'}
<class 'dict'>


In [263]:
import csv
with open('"../movies_dataset.csv"', 'r') as csvfile:
    reader = csv.DictReader(csvfile)


OSError: [Errno 22] Invalid argument: '"../movies_dataset.csv"'